## **Task**

Use your dataset. Write code for the following models:  
Find the minimum risk and corresponding return at the optimal portfolio and compare it with Markowitz models.

* MINMAX

*  MAD model

* VaR model

* CVaR model

In [1]:
import pandas as pd
import numpy as np

num_assets = 50 # number of assets
capital = 100000 # capital to invest

cov_matrix = pd.read_excel('omf_all_data.xlsx', sheet_name='Covariance Matrix', index_col=0)
# fill the upper triangle of the covariance matrix
covariance_matrix = cov_matrix.fillna(cov_matrix.T)
covariance_matrix = covariance_matrix.values

# matrix of 1s
e = np.array([[1] for _ in range(num_assets)])

# expected rate of return
mu = pd.read_excel('omf_all_data.xlsx', sheet_name='Expected rate of return, E(ri)', index_col=0)
mu = mu.values
mu = mu.T

# rate of return
r = pd.read_excel('omf_all_data.xlsx', sheet_name='Rate of Return (ri)', index_col=0)
# drop last row
r = r.drop(r.tail(1).index)
r = r.values

# time period T
T = r.shape[0]

# expected rate of return, R
R = 0.005

## **MINMAX Model**

In [2]:
import gurobipy as gp

# Create a new model
model_minmax = gp.Model("minmax")

# Addition of matrix variable for the stock weights (portfolio)
w = model_minmax.addMVar(num_assets)

# Define variable lambda
l = model_minmax.addMVar(1)

# Set objective function
model_minmax.setObjective(l, gp.GRB.MINIMIZE)

# Constraint: l >= summation over t 1 to T (r_jt - mu_j) * w_j for all j
for j in range(num_assets):
    model_minmax.addConstr(l >= gp.quicksum((r[t, j] - mu[j]) * w[j] for t in range(T)))

# Constraint: l >= - summation over t 1 to T (r_jt - mu_j) * w_j for all j
for j in range(num_assets):
    model_minmax.addConstr(l >= -gp.quicksum((r[t, j] - mu[j]) * w[j] for t in range(T)))

# Constraint: sum(w) = 1
model_minmax.addConstr(gp.quicksum(w) == 1)

# Constraint: mu * w >= R 
model_minmax.addConstr(mu.T @ w >= R)

# Optimize model
model_minmax.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 102 rows, 51 columns and 200 nonzeros
Model fingerprint: 0x80309e40
Coefficient statistics:
  Matrix range     [8e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 100 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 50 columns, 100 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.886878e-01   0.000000e+00      0s
       1    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Opt

In [3]:
# Minimum risk
minmax_risk = model_minmax.objVal

# Optimal weights
weights = w.x

# Optimal portfolio return
portfolio_return = mu.T @ weights

print(f"Minimum risk: {minmax_risk}")
print(f"Optimal weights: {weights}")
print(f"Optimal portfolio return: {portfolio_return}")

Minimum risk: 0.0
Optimal weights: [0.39494723 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.60505277 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
Optimal portfolio return: [0.005]


## **MAD Model**

In [4]:
# Create a new model
model_mad = gp.Model("mad")

# Addition of matrix variable for the stock weights (portfolio)
w = model_mad.addMVar(num_assets)

# Define variable lambda
y = model_mad.addMVar(T)

# Set objective function
model_mad.setObjective(gp.quicksum(y) / T, gp.GRB.MINIMIZE)

# Constraint: yt >= summation over j 1 to n (r_jt - mu_j) * w_j for all t
for t in range(T):
    model_mad.addConstr(y[t] >= gp.quicksum((r[t, j] - mu[j]) * w[j] for j in range(num_assets)))

# Constraint: l >= - summation over j 1 to n (r_jt - mu_j) * w_j for all t
for t in range(T):
    model_mad.addConstr(y[t] >= -gp.quicksum((r[t, j] - mu[j]) * w[j] for j in range(num_assets)))

# Constraint: sum(w) = 1
model_mad.addConstr(gp.quicksum(w) == 1)

# Constraint: mu * w >= R 
model_mad.addConstr(mu.T @ w >= R)

# Optimize model
model_mad.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 992 rows, 545 columns and 50590 nonzeros
Model fingerprint: 0xb62db6c2
Coefficient statistics:
  Matrix range     [3e-07, 1e+00]
  Objective range  [2e-03, 2e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.03s
Presolved: 992 rows, 545 columns, 50590 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 4.915e+05
 Factor NZ  : 4.925e+05 (roughly 5 MB of memory)
 Factor Ops : 3.259e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.13 seconds (0.02 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex

Solved in 584 iterations and 0

In [5]:
# Minimum risk
mad_risk = model_mad.objVal

# Optimal weights
weights = w.x

# Optimal portfolio return
portfolio_return = mu.T @ weights

print(f"Minimum risk: {mad_risk}")
print(f"Optimal weights: {weights}")
print(f"Optimal portfolio return: {portfolio_return}")

Minimum risk: 0.015277525620709165
Optimal weights: [0.05327642 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.08719926 0.         0.         0.23424915
 0.06897883 0.         0.         0.28916118 0.25472714 0.
 0.         0.         0.         0.         0.         0.
 0.01240802 0.         0.         0.         0.         0.
 0.         0.        ]
Optimal portfolio return: [0.005]


## **VaR Model**

In [6]:
from scipy.stats import norm

# Calculate the mean and standard deviation of returns
mean_return = np.mean(r)
std_dev = np.std(r)

# Calculate the VaR at 95% confidence level using the Z-score
confidence_level = 0.95
z_score = norm.ppf(1 - confidence_level)
VaR_variance_covariance = mean_return + z_score * std_dev

print(f"VaR using variance-covariance method: {VaR_variance_covariance}")

VaR using variance-covariance method: -0.040759245435410506


## **CVaR Model**

In [7]:
# Calculate the VaR at 95% confidence level using the historical method
returns = r.flatten()
returns_sorted = np.sort(returns)
VaR_historical = np.percentile(returns_sorted, 5)

print(f"VaR using historical method: {VaR_historical}")

# Calculate the CVaR at 95% confidence level using the historical method
CVaR_historical = np.mean(returns_sorted[returns_sorted <= VaR_historical])

print(f"CVaR using historical method: {CVaR_historical}")

VaR using historical method: -0.0302875106153957
CVaR using historical method: -0.0515080119114732
